In [ ]:
from asyncio import Queue, gather, run
from datetime import datetime

In [ ]:
WORKS = 10

async def log(url):
    with open('log.csv', 'a') as f:
        print(f'Error na url: {url}')
        f.write(f'{datetime.now()};{url}\n')


async def download(url):
    try:
        yt = YouTube(url)
        audio = yt.streams.filter(only_audio=True)[0]
        audio.download()
        print(url)
    except:
        await log(url)


async def get_play_list(url_base):
    plays = Queue()
    playlist = Playlist(url_base)
    for e, url in enumerate(playlist):
        await plays.put((url, e))
    return plays


async def download_videos(play_list):
    while play_list.qsize() != 0:
        print(f'Try download {play_list.qsize()}')
        url = await play_list.get()
        await download(url[0])
    print(f"Finisher")

In [ ]:
async def main():
    PLAYLIST_URL = 'https://www.youtube.com/playlist?list=PL3osQJLUr9gIyTQaJtxT_B75LUWb8WPut'
    play_list = await get_play_list(PLAYLIST_URL)
    print(f'lenght list: {play_list.qsize()}')

    tasks = []
    for _ in range(WORKS):
        tasks.append(create_task(download_videos(play_list)))
    return await gather(*tasks)

In [ ]:
run(main())